# Get Followers & Friends from User Handle using Twython

*Created by Kate Schneider* <br> 
*Last updated June 2, 2020*


#### About the Script

This script is used to download the followers and friends of a user based on either their screen name (handle) or user ID. It uses Twitter's [GET followers/ids](https://developer.twitter.com/en/docs/accounts-and-users/follow-search-get-users/api-reference/get-followers-ids) and [GET friends/ids](https://developer.twitter.com/en/docs/accounts-and-users/follow-search-get-users/api-reference/get-friends-ids) API endpoints.

For this project, I'm using it to download the friends and followers of Canadian MPs (338) and Senators (105).

#### Rate Limits

For both endpoints, the rate limits are as follows:
* 15 requests every 15 min
* no daily or hourly limits

As each request returns 5,000 users, this means you are permitted to download 15,000 users every 15 minutes.

#### Twython Documentation

This test program uses the Twython Python wrapper for Twitter APIs. For documentation on Twython, see [here](https://twython.readthedocs.io/en/latest/index.html).

For a GitHub repository with Twython examples from the creator of Twython, see [here](https://github.com/ryanmcgrath/twython).

### Importing Libraries

The first step is to import Twython, the third party library used to interact with Twitter.

In [ ]:
from twython import Twython, TwythonError

I also have to import 'time' to implement pausing in between requests. 

In [ ]:
import time

### Gaining Access to Twitter

Next, I have to list my user credentials to access the Twitter API. Each authorization key should be inputted as a string.

In [ ]:
access_token = 'ACCESS TOKEN HERE'
access_token_secret = 'SECRET ACCESS TOKEN HERE'
consumer_key = 'CONSUMER KEY HERE'
consumer_secret = 'SECRET CONSUMER KEY HERE'

I have to provide the keys and tokens to Twitter as part of the authorization process and, then, create the actual interface.

In [ ]:
twitter = Twython(consumer_key, consumer_secret, access_token, access_token_secret)

### Variables

Set the following field below to 'True' if trying to get the specified user's followers. Set it to 'False' if trying to get the specified user's friends (who the user follows).

In [ ]:
return_followers = False

Input the users' screen names and/or user IDs into the list below. e.g. ['user1', 'user2', 'user3']

In [ ]:
user_list = ['user1', 'user2']

### Functions

Next, I define the necessary functions.

This function returns a list of user IDs representing the followers of the specified user.

In [ ]:
def followers(user):
    # Setting up an empty list for the follower ids
    follower_ids = []
    
    # Getting the initial batch of the user's followers
    followers = twitter.get_followers_ids(screen_name = user, count = 5000)
    
    # Printing out a statement indicating the request number
    print("Just finished request #1 for user " + user + ".")
    
    # Adding the downloaded followers to the list
    for id in followers['ids']: 
        follower_ids.append(id)
    
    # Setting the request count to 1
    req_num = 1
   
    while followers['next_cursor'] != 0:
        # Pausing for 1 minute to avoid hitting the rate limit
        sleepy()
        
        # Getting a subseqeunt batch of the user's followers
        followers = twitter.get_followers_ids(screen_name = user, count = 5000, cursor = (followers['next_cursor']))                                             
        
        # Adding the downloaded followers to the list 
        for id in followers['ids']: 
            follower_ids.append(id)
        
        # Printing out a statement indicating the request number
        req_num += 1
        print("Just finished request # " + str(req_num) + "for user " + user + ".")
    
    # Printing out a statement indicating requests for user are complete
    print("Finished all requests for user " + user + ".")
    
    return(follower_ids)

This function returns a list of user IDs representing the friends of the specified user.

In [ ]:
def friends(user):
    # Setting up an empty list for the friend ids
    friend_ids = []
    
    # Getting the initial batch of the user's followers
    friends = twitter.get_friends_ids(screen_name = user, count = 5000)
    
    # Printing out a statement indicating the request number
    print("Just finished request #1 for user " + user + ".")
    
    # Adding the downloaded followers to the list
    for id in friends['ids']: 
        friend_ids.append(id)
    
    # Setting the request count to 1
    req_num = 1
   
    while friends['next_cursor'] != 0:
        # Pausing for 1 minute to avoid hitting the rate limit
        sleepy()
        
        # Getting a subseqeunt batch of the user's followers
        friends = twitter.get_followers_ids(screen_name = user, count = 5000, cursor = (followers['next_cursor']))                                             
        
        # Adding the downloaded followers to the list 
        for id in friends['ids']: 
            friend_ids.append(id)
        
        # Printing out a statement indicating the request number
        req_num += 1
        print("Just finished request # " + str(req_num) + "for user " + user + ".")
    
    # Printing out a statement indicating requests for user are complete
    print("Finished all requests for user " + user + ".")
    
    return(friend_ids)

The function below is used to name the .txt file to which the users will be printed.

In [ ]:
def txt_name(user):
    if return_followers == True:
        document = str(user + "_ids_followers.txt")
    else:
        document = str(user + "_ids_friends.txt")
    return(document)

The function below is used to save the user information to a text file.

In [ ]:
def save(results, user):
    with open(txt_name(user), 'w', encoding='utf8') as f:
        print(results, file=f)
    if return_followers == True:
        print("Successfully saved the list of followers of @" + str(user) + "to a .txt file.")
    else:
        print("Successfully saved the list of users followed by @" + str(user) + " to a .txt file.")
    return()

The function below prints out a message based on the error code if a Twython error is raised.

In [ ]:
def error_msg(e, user):
    print("Encountered error when sending request for user @" + str(user) + ".")
    try:
        print(e)
    except:
        print("Unknown error.")
    return()

The function below forces the program to sleep for a minute to avoid tripping the rate limits.

In [ ]:
def sleepy():
    print("Pausing for one minute before completing next request.")
    time.sleep(30)
    print("30 seconds have lapsed. 30 more seconds to go.")
    time.sleep(30)
    print("Finished pausing for one minute. Will proceed to next request.")
    return()

### Driver Code

Now, I run the functions.

In [ ]:
for user in user_list:
    print("\nAttempting requests for user @" + str(user))
    try:
        if return_followers == True:
            # Sending the request to Twitter
            results = followers(user)
            # Saving the results
            save(results, user)
            
        else:
            # Sending the request to Twitter
            results = friends(user)
            # Saving the results
            save(results, user)

    except TwythonError as e:
        # Prints out an error message if an error is raised
        error_msg(e, user)
        break
    
    if user == user_list[-1]:
        # Prints out a finished statement if all of the users have been saved
        print("\nProgram complete! Finished sending requests for each user in the list.")
    else:
        # Pauses the program between users to avoid hitting the rate limit
        sleepy()